In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# Pathway schema and streaming source
fname = "dataset.csv"  # Provided dataset with parking lot events
schema = pw.schema_from_csv(fname)
# Replay the CSV as a stream of events (simulate live data)
data = pw.demo.replay_csv(fname, schema=schema, input_rate=100)

# Pricing parameters (choose base and max prices)
base_price = 5.0
max_price = 15.0

# Baseline linear pricing: price increases linearly with occupancy fraction
def baseline_pricing(occupancy, capacity, base=base_price, max_p=max_price):
    # Linear interpolation between base and max based on occupancy ratio
    return base + (max_p - base) * (occupancy / capacity)

# Demand-based pricing: incorporate occupancy, queue, traffic, special day, vehicle type
def demand_pricing(occupancy, capacity, queue_length, traffic, special_day, vehicle_type, base=base_price):
    occ_frac = occupancy / capacity
    price = base
    # Weight factors (these are illustrative and can be tuned)
    price += 3.0 * occ_frac                    # Occupancy impact
    price += 1.0 * queue_length               # Queue length impact
    # Traffic: map 'low'->1, 'medium'->2, 'high'->3
    traffic_factor = 3.0 if traffic == 'high' else (2.0 if traffic == 'medium' else 1.0)
    price += 1.0 * traffic_factor
    price += 5.0 * special_day                # Special day adds a fixed premium
    # Vehicle type: cars pay more than bikes (for example)
    price += 2.0 if vehicle_type == 'car' else 1.0
    return price

# (Optional) Competitive model stub: could consider nearby lots (not implemented here)
def competitive_pricing(my_price, neighbor_prices):
    # Example: adjust price towards the average of nearby competitors
    if len(neighbor_prices) == 0:
        return my_price
    avg_competitor = np.mean(neighbor_prices)
    # If competitor average is lower, slightly reduce price to compete; if higher, you can increase.
    return my_price * 0.95 if avg_competitor < my_price else my_price * 1.05


In [ ]:

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from collections import defaultdict
import pandas as pd

output_notebook()

# 1. Create sources and figures for all 14 parking lots
plot_sources = {}
plot_figs = {}

for lot_id in range(1, 15):  # assuming lot IDs are 1 to 14
    src = ColumnDataSource(data=dict(time=[], baseline=[], demand=[]))
    fig = figure(
        title=f"Lot {lot_id} - Dynamic Pricing",
        x_axis_type="datetime",
        width=800,
        height=300,
        x_axis_label='Time',
        y_axis_label='Price ($)'
    )
    fig.line('time', 'baseline', source=src, color='blue', legend_label='Baseline')
    fig.line('time', 'demand', source=src, color='red', legend_label='Demand')
    fig.legend.location = "top_left"

    plot_sources[lot_id] = src
    plot_figs[lot_id] = fig

# 2. Show all plots stacked
show(column(*[plot_figs[i] for i in range(1, 15)]))

# Convert traffic and vehicle type to numeric factors in the stream
data = data.with_columns(
    traffic_num = pw.if_else(
        pw.this.TrafficConditionNearby == 'high', 3.0,
        pw.if_else(pw.this.TrafficConditionNearby == 'medium', 2.0, 1.0)
    ),
    vehicle_factor = pw.if_else(pw.this.VehicleType == 'car', 2.0, 1.0)
)

# Compute pricing columns for each incoming event
pricing = data.with_columns(
    Price_Baseline = base_price + (max_price - base_price) * (pw.this.Occupancy / pw.this.Capacity),
    Price_Demand = (base_price
                    + 3.0 * (pw.this.Occupancy / pw.this.Capacity)
                    + 1.0 * pw.this.QueueLength
                    + 1.0 * pw.this.traffic_num
                    + 5.0 * pw.this.IsSpecialDay
                    + 1.0 * pw.this.vehicle_factor
                   )
)

# 3. Define the update function for streaming prices into plots
def update_plot(key, row, time, is_addition):
    if not is_addition:
        return

    try:
        # Safely parse the datetime
        dt = pd.to_datetime(
            row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'],
            format='%d-%m-%Y %H:%M:%S',
            dayfirst=True
        )

        # Get the lot ID (use fallback if missing)
        lot_id = int(row.get('ParkingLotID') or row.get('LotID') or 0)

        # Guard: skip if lot ID not in range
        if lot_id not in plot_sources:
            return

        # Extract price values
        baseline_price = row.get('Price_Baseline')
        demand_price = row.get('Price_Demand')

        # Guard: only update if both are valid
        if baseline_price is None or demand_price is None:
            return

        # Stream to Bokeh plot
        plot_sources[lot_id].stream({
            'time': [dt],
            'baseline': [baseline_price],
            'demand': [demand_price]
        }, rollover=200)

    except Exception as e:
        print("Skipping row due to error:", e)

# 4. Subscribe to Pathway pricing table (stream will push here)
pw.io.subscribe(pricing, update_plot)

In [ ]:
# Convert traffic and vehicle type to numeric factors in the stream
data = data.with_columns(
    traffic_num = pw.if_else(
        pw.this.TrafficConditionNearby == 'high', 3.0,
        pw.if_else(pw.this.TrafficConditionNearby == 'medium', 2.0, 1.0)
    ),
    vehicle_factor = pw.if_else(pw.this.VehicleType == 'car', 2.0, 1.0)
)

# Compute pricing columns for each incoming event
pricing = data.with_columns(
    Price_Baseline = base_price + (max_price - base_price) * (pw.this.Occupancy / pw.this.Capacity),
    Price_Demand = (base_price
                    + 3.0 * (pw.this.Occupancy / pw.this.Capacity)
                    + 1.0 * pw.this.QueueLength
                    + 1.0 * pw.this.traffic_num
                    + 5.0 * pw.this.IsSpecialDay
                    + 1.0 * pw.this.vehicle_factor
                   )
)


In [ ]:
# Prepare Bokeh plot
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

source = ColumnDataSource(data=dict(time=[], baseline=[], demand=[]))
p = figure(title="Parking Pricing over Time", x_axis_type="datetime",
           x_axis_label='Time', y_axis_label='Price',
           width=800, height=400)
p.line('time', 'baseline', source=source, color='blue', legend_label='Baseline')
p.line('time', 'demand', source=source, color='red', legend_label='Demand')
p.legend.location = "top_left"
show(p)

In [ ]:
# Callback to update the plot when a new row (event) arrives
def update_plot(key, row, time, is_addition):
    if is_addition:
        # Combine date and time strings into a pandas datetime
        dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])
        # Stream new data into the plot
        new_data = {
            'time': [dt],
            'baseline': [row['Price_Baseline']],
            'demand': [row['Price_Demand']]
        }
        source.stream(new_data, rollover=200)  # keep last 200 points

# Subscribe to the processed pricing stream
pw.io.subscribe(pricing, update_plot)
# Run the streaming computation (this will begin emitting events and updating the chart)
pw.run()


Output()

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

...

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])

/tmp/ipython-input-9-1493908255.py:5: UserWarning: Parsing dates in %d-%m-%Y %H:%M:%S format when dayfirst=False 
(the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt = pd.to_datetime(row['LastUpdatedDate'] + ' ' + row['LastUpdatedTime'])